In [1]:
# Some imports that might be useful one day
import numpy as np
import urllib.request
import re
import collections
import itertools
import heapq

def get_input(day):
  return urllib.request.urlopen(f'https://raw.githubusercontent.com/SantoSimone/Advent-of-Code/master/2020/input_files/input{day}.txt').read().decode('utf-8')

def parse_ints(text):
  return [int(x) for x in re.findall(r'\d+', text)]